In [38]:
import numpy as np
import pandas as pd


In [39]:
# Leitura do dataset (usamos, agora, a base limpa gerada na etapa 1 do projeto)
df = pd.read_csv('dados_limpos.csv')

Realiza primeiro o processo de encoding em colunas binárias para facilitar na modelagem

In [40]:
def encoding_binary_columns(column, map_values):
    
    if column not in df.columns:
        print(f"Erro, coluna {column} não encontrada no dataframe")
    
    print(f"Convertendo coluna: '{column}'...")
    print(f"  Valores ANTES: {df[column].unique()}")
    # Aplica mapeamento na coluna
    df[column] = df[column].map(map_values)
    print(f"Valores DEPOIS: {df[column].unique()}\n")
    

encoding_binary_columns('Attrition', {'yes': 1, 'no': 0})
encoding_binary_columns('OverTime', {'yes': 1, 'no': 0})
encoding_binary_columns('Gender', {'female': 1, 'male': 0})


Convertendo coluna: 'Attrition'...
  Valores ANTES: ['yes' 'no']
Valores DEPOIS: [1 0]

Convertendo coluna: 'OverTime'...
  Valores ANTES: ['yes' 'no']
Valores DEPOIS: [1 0]

Convertendo coluna: 'Gender'...
  Valores ANTES: ['female' 'male']
Valores DEPOIS: [1 0]



In [41]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears_log,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany_log,YearsInCurrentRole_log,YearsSinceLastPromotion_log,YearsWithCurrManager
0,41,1,travel_rarely,1102,sales,1.0,2,life sciences,2.0,1,...,3.0,1,0,2.197225,0,1,1.945910,1.609438,0.000000,5
1,49,0,travel_frequently,279,research & development,8.0,1,life sciences,3.0,0,...,3.0,4,1,2.397895,3,3,2.397895,2.079442,0.693147,7
2,37,1,travel_rarely,1373,research & development,2.0,2,other,3.0,0,...,3.0,2,0,2.079442,3,3,0.000000,0.000000,0.000000,0
3,33,0,travel_frequently,1392,research & development,7.0,4,life sciences,4.0,1,...,3.0,3,0,2.197225,3,3,2.197225,2.079442,1.386294,0
4,27,0,travel_rarely,591,research & development,2.0,1,medical,1.0,0,...,3.0,4,1,1.945910,3,3,1.098612,1.098612,1.098612,2
